In [1]:
import sys

f = open('C:/Users/jedgo/Projects/Jupyter1/tf/reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('C:/Users/jedgo/Projects/Jupyter1/tf/labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i;


input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [2]:
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

alpha, iterations = (0.01, 3)
hidden_size, next_size = (100, 100)

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,next_size)) - 0.1
weights_2_3 = 0.2*np.random.random((next_size,1)) - 0.1

correct,total = (0,0)
for iter in range(iterations):

    for i in range(len(input_dataset)-1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
        layer_3 = sigmoid(np.dot(layer_2,weights_2_3))

        layer_3_delta = layer_3 - y
        layer_2_delta = layer_3_delta.dot(weights_2_3.T)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha
        weights_2_3 -= np.outer(layer_2,layer_3_delta) * alpha

        if(np.abs(layer_3_delta) < 0.5 ):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter: ' + str(iter) + ' Progress:' + progress[2:4] + '.' + progress[4:6] + '% Training Accuracy: ' + str(correct/float(total)) + '%')
    print()
correct, total = (0,0)
for i in range(len(input_dataset) - 1000, len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    layer_3 = sigmoid(np.dot(layer_2,weights_2_3))

    if(np.abs(layer_3 - y) < 0.5 ):
        correct += 1
    total += 1
print("Test Accuracy: " + str(correct / float(total)))

Iter: 0 Progress:95.99% Training Accuracy: 0.78725%
Iter: 1 Progress:95.99% Training Accuracy: 0.8432291666666667%
Iter: 2 Progress:95.99% Training Accuracy: 0.8698888888888889%
Test Accuracy: 0.851


In [7]:
from collections import Counter
import math

def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word, index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10)

print(similar('beautiful'))
print(similar('terrible'))
print(similar('smart'))

[('beautiful', -0.0), ('impressive', -0.7148938567730739), ('bit', -0.7394569305187151), ('prince', -0.740014458931044), ('haunting', -0.7455040941615296), ('certain', -0.7478191195430113), ('best', -0.7592300404830367), ('accurate', -0.7682703703342567), ('missed', -0.7697537284891116), ('plenty', -0.7700248835376932)]
[('terrible', -0.0), ('worse', -0.803741466989269), ('dull', -0.8457500723301327), ('mess', -0.8465227340412378), ('avoid', -0.8564634348812277), ('laughable', -0.8566769561749996), ('fails', -0.857409794280264), ('annoying', -0.9066030651906202), ('badly', -0.9123456941614194), ('horrible', -0.9218620802590545)]
[('smart', -0.0), ('suspenseless', -0.6384205358427958), ('polarizes', -0.6544944275514023), ('bas', -0.6570683989430544), ('stucco', -0.6616703930920318), ('desultory', -0.664626779337941), ('demonstrable', -0.6666043155701503), ('fictional', -0.6667992558135957), ('redline', -0.6690010583325732), ('ryna', -0.6690327481264424)]


In [9]:
import sys,random,math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)
f = open('C:/Users/jedgo/Projects/Jupyter1/tf/reviews.txt')
raw_reviews = f.readlines()
f.close()

tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i

concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)

random.shuffle(input_dataset)

alpha, iterations = (0.05, 2)
hidden_size, window, negative = (50,2,5)

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.05) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size) * 0.2

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

# def similar(target='beautiful'):
#   target_index = word2index[target]
#
#    scores = Counter()
#    for word,index in word2index.items():
#        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
#        squared_difference = raw_difference * raw_difference
#        scores[word] = -math.sqrt(sum(squared_difference))
#    return scores.most_common(10)

def analogy(positive=['terrible','good'],negative=['bad']):

    norms = np.sum(weights_0_1 * weights_0_1,axis=1)
    norms.resize(norms.shape[0],1)

    normed_weights = weights_0_1 * norms

    query_vect = np.zeros(len(weights_0_1[0]))
    for word in positive:
        query_vect += normed_weights[word2index[word]]
    for word in negative:
        query_vect -= normed_weights[word2index[word]]

    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - query_vect
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10)[1:]

def sigmoid(x):
    return 1/(1 + np.exp(-x))

for rev_i,review in enumerate(input_dataset * iterations):
    for target_i in range(len(review)):

        target_samples = [review[target_i]] + list(concatenated[(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])

        left_context = review[max(0,target_i-window):target_i]
        right_context = review[target_i+1:min(len(review),target_i+window)]

        layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
        layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
        layer_2_delta = layer_2 - layer_2_target
        layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

        weights_0_1[left_context + right_context] -= layer_1_delta * alpha
        weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1) * alpha

    if(rev_i % 250 == 0):
        sys.stdout.write('\rProgress: '+str(rev_i/float(len(input_dataset) * iterations)) + "    " + str(similar('terrible')))
    sys.stdout.write('\rProgress: '+str(rev_i/float(len(input_dataset) * iterations)))
print(similar('terrible'))
print(similar('beautiful'))

Progress: 0.99998[('terrible', -0.0), ('horrible', -2.839337935366001), ('brilliant', -3.434764577755796), ('pathetic', -3.6675122526431516), ('marvelous', -3.769788224748615), ('superb', -3.99032955554551), ('phenomenal', -3.993463014007861), ('bad', -4.028620092196871), ('mediocre', -4.105461167977133), ('masterful', -4.201211352365427)]
[('beautiful', -0.0), ('lovely', -3.2940812983844676), ('creepy', -3.4827060637279357), ('glamorous', -3.491098450506346), ('fantastic', -3.497190110158129), ('spooky', -3.5532883391090837), ('nightmarish', -3.5926771434101634), ('gorgeous', -3.616151996020838), ('phenomenal', -3.617936712584124), ('cute', -3.6431917268012586)]
